<a href="https://colab.research.google.com/github/Shamita19/Hyperdimensional_Computing/blob/main/LBP%2BHDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob            # for file locations
import pprint          # for pretty printing
import re

pp = pprint.PrettyPrinter()

def file_list(folder_path, output=False):
    # create an empty list
    file_list = []
    # for file name in the folder path...
    for filename in glob.glob(folder_path):
        # ... append it to the list
        file_list.append(filename)

    # sort alphabetically
    file_list.sort()

    # Output
    if output:
        print(str(len(file_list)) + " files found")
        pp.pprint(file_list)

    return file_list

In [ ]:
import os
import re


upenn_ictal_list = []
upenn_interictal_list = []

# General data directory
UPENN_DATA_DIR = '/content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips'
# Iterate over each patient folder (Patient_1 to Patient_5)
for patient_id in range(1,6 ):
    patient_folder = f"Patient_{patient_id}"
    patient_data_dir = os.path.join(UPENN_DATA_DIR, patient_folder)
    #print(patient_data_dir)
    # get folder lists
    patient_file_list = file_list(os.path.join(patient_data_dir, '*'), output=False)
    #print(patient_file_list)
    for file in patient_file_list:
      if re.findall('interictal', file):
        upenn_interictal_list.append(file)
      elif re.findall('ictal', file):
        upenn_ictal_list.append(file)
print(len(upenn_ictal_list))

print(len(upenn_interictal_list))

220
30


In [ ]:
import os
import re


upenn_test = []

# General data directory
UPENN_DATA_DIR = '/content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips'
# Iterate over each patient folder (Patient_1 to Patient_5)
for patient_id in range(6,7 ):
    patient_folder = f"Patient_{patient_id}"
    patient_data_dir = os.path.join(UPENN_DATA_DIR, patient_folder)
    #print(patient_data_dir)
    # get folder lists
    patient_file_list = file_list(os.path.join(patient_data_dir, '*'), output=False)
    #print(patient_file_list)
    for file in patient_file_list:
      if re.findall('.mat', file):
        upenn_test.append(file)

In [ ]:
import torch
import numpy as np
import math
class HD_classifier:
    def __init__(self, N_seats1, HD_dim, N_seats2, T, string):
        self.training_done = False
        self.N_seats1 = N_seats1
        self.N_seats2 = N_seats2
        self.HD_dim = HD_dim
        self.T = T
        #self.device = device
        #self.cuda = cuda
        self.mat_LBP= torch.randn(self.HD_dim, self.N_seats1)
        self.mat_LBP[self.mat_LBP >0] = 1
        self.mat_LBP[self.mat_LBP <= 0] = 0
        self.mat_channels= torch.randn(self.HD_dim, self.N_seats2)
        self.mat_channels[self.mat_channels >=0] = 1
        self.mat_channels[self.mat_channels < 0] = 0

    def learn(self, EEG):
        HVector = torch.ShortTensor(1,10000).zero_()
        N_channels, learningEnd = EEG.size()
        #print(learningEnd)
        LBP_weights = torch.ShortTensor([2**0, 2**1, 2**2])
        for i in range(learningEnd-3):
            #print(EEG[:, i:(i+self.T)])
            x = EEG[:, i:(i+self.T)].short()
            #print('x',x)

            bp = (torch.add(-x[:, 0:self.T-1], 1, x[:, 1:self.T]) > 0).short()
            #print('bp',bp.shape)
            value = torch.sum(torch.mul(LBP_weights, bp), dim=1)
            #print('value',value.shape)
            binding_vector = self.xor(self.mat_channels, self.mat_LBP[:, value.long()])
            #print('bv',binding_vector.shape)
            output_vector = torch.sum(binding_vector, dim=1)
            #print('op',output_vector)
            if N_channels % 2 == 0:
                output_vector = torch.add(self.xor(self.mat_LBP[:, 1], self.mat_LBP[:, 2]), 1, output_vector)
            output_vector = (output_vector > int(math.floor(self.N_seats2/2))).short()
            HVector=torch.add(HVector,1,output_vector)
            #print('vector',HVector)
        HVector = (HVector > (learningEnd-6)/2).short()
        #print('vector',HVector)
        return HVector

    def learn_big(self, EEG, fs):
        HVector = torch.ShortTensor(1,10000).zero_()
        #HVector = torch.ShortTensor(1, self.mat_channels.size(1)).zero_()
        N_channels, learningEnd = EEG.size()
        index = np.arange(0, learningEnd, float(fs/2))
        #index = np.arange(0, learningEnd,0.5)
        #print('i',index)
        for i in index:
            temp = self.learn(EEG[:, int(i):int(i+fs/2)])
            #HVector[:, :temp.size(1)] += temp
            HVector += temp
        HVector = (HVector > (index.size/2)).short()
        return HVector

    def predict(self, testVector, Ictalprot, Interictalprot, D):
        distanceVector0 = self.ham_dist(testVector, Ictalprot, D)
        #print('distanceVector0',distanceVector0)
        distanceVector1 = self.ham_dist(testVector, Interictalprot, D)
        #print('distanceVector1',distanceVector1)
        if distanceVector0 > distanceVector1:
            prediction = 0
        else:
            prediction = 1
        print('prediction',prediction)
        return distanceVector0, distanceVector1, prediction

    def xor(self, vec_a, vec_b):
        #print('vec a',vec_a)
        #print('vec b',vec_b)
        vec_c = (torch.add(vec_a, vec_b) == 1).short()
        return vec_c

    def ham_dist(self, vec_a, vec_b, D):
        vec_c = self.xor(vec_a, vec_b)
        dist = torch.sum(vec_c) / int(D)
        return dist


In [ ]:
import time

fs= 256
second = fs
minutes = second*60
T = 7	#1 + dimension l of binary pattern
totalNumberBP = 2**(T-1)
D = 10000   # dimension of hypervector
#List of patient that we want to test
Patient_list= [4]
Patient_trans = [1, 2, 3, 4, 5, 6]
crossValidationK = [4, 4, 1, 4, 6, 2]
Seizuretest = [5, 4, 14, 1, 6, 2]
Seizuretrain= [2, 1, 3, 1, 1, 1]
data_dir = "/content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips"
for Patient in Patient_list:
  EEGSez,label= load_data("ictal",Patient_trans[Patient-1])
  nSignals = EEGSez.size(0)
  model = HD_classifier(totalNumberBP,D,nSignals,T, 'random')
  for cross in range(crossValidationK[Patient-1]):
    for i in range(Seizuretrain[Patient-1]):
      EEGSez,label = load_data("ictal",Patient)
      print ('Learning Seizure number ' + str(i))
      temp1=model.learn_big(EEGSez,fs)
      if i == 0:
        queryV0 = temp1
      else:
        queryV0 = torch.add(queryV0,temp1)
    queryV0 = (queryV0>Seizuretrain[Patient-1]/2).short()
    EEGInterictal,label = load_data("interictal",Patient)
    print('Learning Interictal period')
    temp3=model.learn_big(EEGInterictal,fs)
    queryV1 = temp3
    i = 0
    j = 0
    t = time.time()
    for i in range(Seizuretest[Patient-1]):
      filepath = os.path.join(data_dir, f"Patient_{Patient}")
      patient_file_list = os.listdir(filepath)
      print('path',os.path.join(filepath, patient_file_list[i]))
      f = scp.loadmat(os.path.join(filepath, patient_file_list[i]))
      distanceV0 = torch.zeros(1,int(f['data'].shape[0]/fs*2))
      print('shape',distanceV0.size(dim=1))
      distanceV1 = torch.zeros(1,int(f['data'].shape[0]/fs*2))
      prediction0 = torch.zeros(1,int(f['data'].shape[0]/fs*2))
      EEGtest=torch.from_numpy(np.array(f['data'])).t()
      index = np.arange(0,EEGtest.size(1),fs/2)
      i = 0
      j = j+1
      for iStep in index:
        temp = model.learn(EEGtest[:,int(iStep):int(iStep+fs/2)])
        [distanceV0[i],distanceV1[i],prediction0[i]] = model.predict(temp, queryV0, queryV1, D)
        i = i + 1
        print ('Patient:' +str(Patient)+ 'K-fold:' + str(cross+1)+'Seizure: ' + str(j) +'; half second: ' + str(i))

4
label 1
4
label 1
Learning Seizure number 0
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.int16)
vector tensor([[1]], dtype=torch.i

In [ ]:
import os
import scipy.io as scp
import numpy as np
import torch
import random

data_dir = "/content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips"

def load_data(seg_type, num):
    print(num)
    filepath = os.path.join(data_dir, f"Patient_{num}")
    patient_file_list = os.listdir(filepath)
    lab = labels(seg_type)
    if lab == 0:
        patient_file = random.choice([f for f in patient_file_list if 'ictal' in f])
    elif lab == 1:
        patient_file = random.choice([f for f in patient_file_list if 'interictal' in f])

    f = scp.loadmat(os.path.join(filepath, patient_file))
    eeg_data = torch.from_numpy(np.array(f['data'])).t()
    print('label', lab)
    return eeg_data, lab

def labels(segment_type):
    if segment_type == "ictal":
        return 0
    elif segment_type == "interictal":
        return 1
    else:
        raise ValueError("Invalid segment type")


In [ ]:
import time

fs= 250
second = fs
minutes = second*60
T = 4	#1 + dimension l of binary pattern
totalNumberBP = 2**(T-1)
D = 10000   # dimension of hypervector
#List of patient that we want to test
Patient_list= [4]
Patient_trans = [1, 2, 3, 4, 5, 6, 7, 8]
#crossValidationK = [4, 4, 1, 4, 6, 2]
Seizuretest = [5, 4, 4, 47, 6, 2, 5, 5]
Seizuretrain= [2, 1, 3, 3, 1, 1, 1, 2]
data_dir = "/content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips"

for Patient in Patient_list:
  acc = np.zeros(Seizuretest[Patient-1])
  EEGSez,label= load_data("ictal",Patient)
  nSignals = EEGSez.size(0)
  torch.manual_seed(0)
  model = HD_classifier(totalNumberBP,D,nSignals,T, 'random')
  for i in range(Seizuretrain[Patient-1]):
      EEGSez,label = load_data("ictal",Patient)
      print ('Learning Seizure number ' + str(i))
      temp1=model.learn_big(EEGSez,fs)
      if i == 0:
        queryV0 = temp1
        print('queryV0',queryV0)
      else:
        queryV0 = torch.add(queryV0,temp1)
        print('queryV0',queryV0)
      EEGInterictal,label = load_data("interictal",Patient)
      print('Learning Interictal period')
      temp3=model.learn_big(EEGInterictal,fs)
      if i == 0:
        queryV1 = temp3
        print('queryV1',queryV1)
      else:
        queryV1 = torch.add(queryV1,temp3)
        print('queryV1',queryV1)
  queryV0 = (queryV0 > Seizuretrain[Patient-1]/2).short()
  print('queryV0',queryV0)
  queryV1 = (queryV1 > Seizuretrain[Patient-1]/2).short()
  print('queryV1',queryV1)
  #EEGInterictal,label = load_data("interictal",Patient)
  #print('Learning Interictal period')
  #temp3=model.learn_big(EEGInterictal,fs)
  #queryV1 = temp3
  #print('queryV1',queryV1)
  i = 0
  j = 0
  t = time.time()
  for i in range(Seizuretest[Patient-1]):
      filepath = os.path.join(data_dir, f"Patient_{Patient}")
      patient_file_list = os.listdir(filepath)
      print('path',os.path.join(filepath, patient_file_list[i]))
      f = scp.loadmat(os.path.join(filepath, patient_file_list[i]))
      EEGtest=torch.from_numpy(np.array(f['data'])).t()
      actual_labels = []
      la= check_path(patient_file_list[i])
      actual_labels.append(la)
      print(actual_labels)
      index = np.arange(0,EEGtest.size(1),float(fs/2))
      #print(EEGtest.size(1))
      #print(index)
      k = 0
      j = j+1
      for iStep in index:
        temp = model.learn(EEGtest[:,int(iStep):int(iStep+fs/2)])
        distanceV0,distanceV1,prediction0 = model.predict(temp, queryV0, queryV1, D)
        print(distanceV0,distanceV1,prediction0)
        k = k + 1
        print ('Patient:' +str(Patient) +'Seizure: ' + str(j) +'; half second: ' + str(i))

        correct_predictions = (prediction0 == torch.tensor(actual_labels[0])).sum().item()
        #total_predictions = 0
        #correct_predictions = (prediction0[k] == (actual_labels[k])).sum()
        total_predictions = len(actual_labels)
        accuracy = correct_predictions / total_predictions
        print(f"Accuracy for Patient {Patient}, Seizure {j}: {accuracy*100}")
        acc[i] = accuracy


mean = acc.mean()
std = acc.std()
print(f"Mean accuracy: {mean*100}")
print(f"Standard deviation: {std*100}")


4
label 0
4
label 0
Learning Seizure number 0
queryV0 tensor([[0, 0, 0,  ..., 1, 1, 1]], dtype=torch.int16)
4
label 1
Learning Interictal period
queryV1 tensor([[1, 0, 1,  ..., 1, 1, 1]], dtype=torch.int16)
4
label 0
Learning Seizure number 1
queryV0 tensor([[0, 1, 0,  ..., 2, 2, 2]], dtype=torch.int16)
4
label 1
Learning Interictal period
queryV1 tensor([[1, 0, 2,  ..., 2, 2, 2]], dtype=torch.int16)
4
label 0
Learning Seizure number 2
queryV0 tensor([[0, 1, 0,  ..., 3, 3, 3]], dtype=torch.int16)
4
label 1
Learning Interictal period
queryV1 tensor([[1, 1, 2,  ..., 3, 3, 3]], dtype=torch.int16)
queryV0 tensor([[0, 0, 0,  ..., 1, 1, 1]], dtype=torch.int16)
queryV1 tensor([[0, 0, 1,  ..., 1, 1, 1]], dtype=torch.int16)
path /content/drive/MyDrive/ECE 556 PROJECT/Volumes/Seagate/seizure_detection/competition_data/clips/Patient_4/Patient_4_interictal_segment_12.mat
[1]
prediction 1
tensor(0.2071) tensor(0.2115) 1
Patient:4Seizure: 1; half second: 0
Accuracy for Patient 4, Seizure 1: 100.0
pa

In [ ]:
def check_path(path):
    if 'interictal' in path:
        return 1
    else:
        return 0
